In [ ]:
from bs4 import BeautifulSoup, Comment
from urllib.request import urlopen
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Temporadas analisadas
seasons = [1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989,
           1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999,
           2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009,
           2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
           2020]

In [ ]:
# Dicionário de nomes completos dos times e suas abreviaturas
teams = {
    'Atlanta Hawks': 'ATL',
    'Boston Celtics': 'BOS',
    'Brooklyn Nets': 'BRK',
    'Charlotte Bobcats': 'CHA',
    'Charlotte Hornets': 'CHH/CHO',
    'Chicago Bulls': 'CHI',
    'Cleveland Cavaliers': 'CLE',
    'Dallas Mavericks': 'DAL',
    'Denver Nuggets': 'DEN',
    'Detroit Pistons': 'DET',
    'Golden State Warriors': 'GSW',
    'Houston Rockets': 'HOU',
    'Indiana Pacers': 'IND',
    'Kansas City Kings': 'KCK',
    'Los Angeles Clippers': 'LAC',
    'Los Angeles Lakers': 'LAL',
    'Memphis Grizzlies': 'MEM',
    'Miami Heat': 'MIA',
    'Milwaukee Bucks': 'MIL',
    'Minnesota Timberwolves': 'MIN',
    'New Jersey Nets': 'NJN',
    'New Orleans Hornets': 'NOH',
    'New Orleans/Oklahoma City Hornets': 'NOK',
    'New Orleans Pelicans': 'NOP',
    'New York Knicks': 'NYK',
    'Oklahoma City Thunder': 'OKC',
    'Orlando Magic': 'ORL',
    'Philadelphia 76ers': 'PHI',
    'Phoenix Suns': 'PHO',
    'Portland Trail Blazers': 'POR',
    'Sacramento Kings': 'SAC',
    'San Antonio Spurs': 'SAS',
    'San Diego Clippers': 'SDC',
    'Seattle SuperSonics': 'SEA',
    'Toronto Raptors': 'TOR',
    'Utah Jazz': 'UTA',
    'Vancouver Grizzlies': 'VAN',
    'Washington Wizards': 'WAS',
    'Washington Bullets': 'WSB'
}

In [ ]:
# Scrap Standings
standings_per_season = []
for season in seasons:
    url = 'https://www.basketball-reference.com/leagues/NBA_{}_standings.html'.format(season)
    html = urlopen(url)
    soup = BeautifulSoup(html)

    # Obter código comentado
    commented_html = []
    for comments in soup.findAll(text = lambda text:isinstance(text, Comment)):
        commented_tag = comments.extract()
        commented_html.append(commented_tag)
    
    commented_soup = BeautifulSoup(commented_html[26])

    # Obter cabeçalho
    headers = [th.getText() for th in commented_soup.findAll('tr', limit = 2)[1].findAll('th')]
    headers = headers[0:2]

    # Obter linhas
    rows = commented_soup.findAll('tr')[2:]
    team_stats = [[td.getText() for td in rows[i].findAll(lambda tag: tag.has_attr('data-stat') and tag['data-stat'] == 'ranker' or 'team_name')][:2] for i in range(len(rows))]
    stats = pd.DataFrame(team_stats, columns = headers)
    stats['Season'] = season

    # Adicionar dados a lista
    standings_per_season.append(stats)

# Unir todos os dados da lista
standings_stats = pd.concat(standings_per_season)

In [ ]:
# Converter nomes completos dos times para o abreviado
def transform_variable_Team(x: str) -> str:
    return teams[x]

standings_stats.Team = standings_stats.Team.apply(transform_variable_Team)

maskChh = (standings_stats.Team == 'CHH/CHO') & (standings_stats.Season <= 2002)
standings_stats.Team[maskChh] = 'CHH'

maskCho = (standings_stats.Team == 'CHH/CHO') & (standings_stats.Season >= 2015)
standings_stats.Team[maskCho] = 'CHO'

In [ ]:
standings_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1134 entries, 0 to 29
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Rk      1134 non-null   object
 1   Team    1134 non-null   object
 2   Season  1134 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 35.4+ KB


In [ ]:
# Scrap Basic Stats
basic_stats_per_season = []
for season in seasons:
    url = 'https://www.basketball-reference.com/leagues/NBA_{}_per_game.html'.format(season)
    html = urlopen(url)
    soup = BeautifulSoup(html)

    # Obter cabeçalho
    headers = [th.getText() for th in soup.findAll('tr', limit = 2)[0].findAll('th')]
    headers = headers[1:]

    # Obter linhas
    rows = soup.findAll('tr', class_ = lambda x: x != 'thead')[1:]
    player_stats = [[td.getText() for td in rows[i].findAll('td')] for i in range(len(rows))]
    stats = pd.DataFrame(player_stats, columns = headers)
    stats['Season'] = season

    # Adicionar dados a lista
    basic_stats_per_season.append(stats)

# Unir todos os dados da lista
basic_stats = pd.concat(basic_stats_per_season)

In [ ]:
basic_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20950 entries, 0 to 650
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Player  20950 non-null  object
 1   Pos     20950 non-null  object
 2   Age     20950 non-null  object
 3   Tm      20950 non-null  object
 4   G       20950 non-null  object
 5   GS      20950 non-null  object
 6   MP      20950 non-null  object
 7   FG      20950 non-null  object
 8   FGA     20950 non-null  object
 9   FG%     20950 non-null  object
 10  3P      20950 non-null  object
 11  3PA     20950 non-null  object
 12  3P%     20950 non-null  object
 13  2P      20950 non-null  object
 14  2PA     20950 non-null  object
 15  2P%     20950 non-null  object
 16  eFG%    20950 non-null  object
 17  FT      20950 non-null  object
 18  FTA     20950 non-null  object
 19  FT%     20950 non-null  object
 20  ORB     20950 non-null  object
 21  DRB     20950 non-null  object
 22  TRB     20950 non-null  

In [ ]:
# Remover colunas desnecessárias
basic_stats.drop(columns = ['FG', 'FGA', '3P', '3PA', '2P', '2PA', 'FT', 'FTA', 'ORB', 'DRB'], inplace = True)

In [ ]:
basic_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20950 entries, 0 to 650
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Player  20950 non-null  object
 1   Pos     20950 non-null  object
 2   Age     20950 non-null  object
 3   Tm      20950 non-null  object
 4   G       20950 non-null  object
 5   GS      20950 non-null  object
 6   MP      20950 non-null  object
 7   FG%     20950 non-null  object
 8   3P%     20950 non-null  object
 9   2P%     20950 non-null  object
 10  eFG%    20950 non-null  object
 11  FT%     20950 non-null  object
 12  TRB     20950 non-null  object
 13  AST     20950 non-null  object
 14  STL     20950 non-null  object
 15  BLK     20950 non-null  object
 16  TOV     20950 non-null  object
 17  PF      20950 non-null  object
 18  PTS     20950 non-null  object
 19  Season  20950 non-null  int64 
dtypes: int64(1), object(19)
memory usage: 3.4+ MB


In [ ]:
# Scrap Advanced Stats
advanced_stats_per_season = []
for season in seasons:
    url = 'https://www.basketball-reference.com/leagues/NBA_{}_advanced.html'.format(season)
    html = urlopen(url)
    soup = BeautifulSoup(html)

    # Obter cabeçalh
    headers = [th.getText() for th in soup.findAll('tr', limit = 2)[0].findAll('th')]
    headers = headers[1:]

    # Obter linhas
    rows = soup.findAll('tr', class_ = lambda x: x != 'thead')[1:]
    player_stats = [[td.getText() for td in rows[i].findAll('td')] for i in range(len(rows))]
    stats = pd.DataFrame(player_stats, columns = headers)
    stats['Season'] = season

    # Adicionar dados a lista
    advanced_stats_per_season.append(stats)

# Unir todos os dados da lista
advanced_stats = pd.concat(advanced_stats_per_season)

In [ ]:
advanced_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20950 entries, 0 to 650
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Player  20950 non-null  object
 1   Pos     20950 non-null  object
 2   Age     20950 non-null  object
 3   Tm      20950 non-null  object
 4   G       20950 non-null  object
 5   MP      20950 non-null  object
 6   PER     20950 non-null  object
 7   TS%     20950 non-null  object
 8   3PAr    20950 non-null  object
 9   FTr     20950 non-null  object
 10  ORB%    20950 non-null  object
 11  DRB%    20950 non-null  object
 12  TRB%    20950 non-null  object
 13  AST%    20950 non-null  object
 14  STL%    20950 non-null  object
 15  BLK%    20950 non-null  object
 16  TOV%    20950 non-null  object
 17  USG%    20950 non-null  object
 18          20950 non-null  object
 19  OWS     20950 non-null  object
 20  DWS     20950 non-null  object
 21  WS      20950 non-null  object
 22  WS/48   20950 non-null  

In [ ]:
# Remover colunas desnecessárias
advanced_stats.drop(columns = ['\xa0', 'Pos', 'Age', 'G', 'MP', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%',
                               'AST%', 'STL%', 'BLK%', 'TOV%', 'OWS', 'DWS', 'OBPM', 'DBPM'], inplace = True)

In [ ]:
advanced_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20950 entries, 0 to 650
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Player  20950 non-null  object
 1   Tm      20950 non-null  object
 2   PER     20950 non-null  object
 3   TS%     20950 non-null  object
 4   USG%    20950 non-null  object
 5   WS      20950 non-null  object
 6   WS/48   20950 non-null  object
 7   BPM     20950 non-null  object
 8   VORP    20950 non-null  object
 9   Season  20950 non-null  int64 
dtypes: int64(1), object(9)
memory usage: 1.8+ MB


In [ ]:
# Scrap MVP Voting
mvp_voting_per_season = []
for season in seasons:
    url = 'https://www.basketball-reference.com/awards/awards_{}.html'.format(season)
    html = urlopen(url)
    soup = BeautifulSoup(html)

    # Obter cabeçalho
    headers = [th.getText() for th in soup.findAll('tr', limit = 2)[1].findAll('th')]
    headers = headers[1:]

    # Obter linhas
    table = soup.find(lambda tag: tag.has_attr('id') and tag['id'] == 'mvp')
    rows = table.findAll('tr', class_ = lambda x: x != 'thead')[1:]
    player_stats = [[td.getText() for td in rows[i].findAll('td')] for i in range(len(rows))]
    stats = pd.DataFrame(player_stats, columns = headers)
    stats['Season'] = season

    # Adicionar dados a lista
    mvp_voting_per_season.append(stats)

# Unir todos os dados da lista
mvp_voting = pd.concat(mvp_voting_per_season)

In [ ]:
mvp_voting.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 711 entries, 0 to 12
Data columns (total 20 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Player   670 non-null    object
 1   Age      670 non-null    object
 2   Tm       670 non-null    object
 3   First    670 non-null    object
 4   Pts Won  670 non-null    object
 5   Pts Max  670 non-null    object
 6   Share    670 non-null    object
 7   G        670 non-null    object
 8   MP       670 non-null    object
 9   PTS      670 non-null    object
 10  TRB      670 non-null    object
 11  AST      670 non-null    object
 12  STL      670 non-null    object
 13  BLK      670 non-null    object
 14  FG%      670 non-null    object
 15  3P%      670 non-null    object
 16  FT%      670 non-null    object
 17  WS       670 non-null    object
 18  WS/48    670 non-null    object
 19  Season   711 non-null    int64 
dtypes: int64(1), object(19)
memory usage: 116.6+ KB


In [ ]:
# Remover colunas desnecessárias
mvp_voting.drop(columns = ['Age', 'Pts Max', 'G', 'MP', 'PTS', 'TRB', 'AST',
                           'STL', 'BLK', 'FG%', '3P%', 'FT%', 'WS', 'WS/48'], inplace = True)

In [ ]:
mvp_voting.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 711 entries, 0 to 12
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Player   670 non-null    object
 1   Tm       670 non-null    object
 2   First    670 non-null    object
 3   Pts Won  670 non-null    object
 4   Share    670 non-null    object
 5   Season   711 non-null    int64 
dtypes: int64(1), object(5)
memory usage: 38.9+ KB


In [ ]:
# Converter nomes completos dos jogadores aposentados (remover *)
def transform_variable_Player(x: str) -> str:
    return x.replace('*', '')

basic_stats.Player = basic_stats.Player.apply(transform_variable_Player)
advanced_stats.Player = advanced_stats.Player.apply(transform_variable_Player)

# Data frame final
df = basic_stats.copy()

# Criar índices únicos
df['PlayerTmSeason'] = df['Player'].map(str) + df['Tm'].map(str) + df['Season'].map(str)
df['TmSeason'] = df['Tm'].map(str) + df['Season'].map(str)

advanced_stats['PlayerTmSeason'] = advanced_stats['Player'].map(str) + advanced_stats['Tm'].map(str) + advanced_stats['Season'].map(str)
mvp_voting['PlayerTmSeason'] = mvp_voting['Player'].map(str) + mvp_voting['Tm'].map(str) + mvp_voting['Season'].map(str)
standings_stats['TmSeason'] = standings_stats['Team'].map(str) + standings_stats['Season'].map(str)

# Unificar data frames
df = df.merge(advanced_stats, on = 'PlayerTmSeason', how = 'left')
df = df.merge(mvp_voting, on = 'PlayerTmSeason', how = 'left')
df = df.merge(standings_stats, on = 'TmSeason', how = 'left')

# Remover colunas duplicada
df = df.loc[:, ~df.columns.duplicated()]
df.drop(columns = ['PlayerTmSeason', 'TmSeason', 'Player_y', 'Tm_y', 'Season_y', 'Player', 'Tm', 'Team'], inplace = True)

# Renomear colunas
df.rename(columns = {'Player_x': 'Player', 'Tm_x': 'Team', 'Season_x': 'Season', 'Rk': 'Team Overall Seed'}, inplace = True)

# Preencher valores nulos
df.fillna({'GS': 0, 'PER': 0, 'USG%': 0, 'First': 0, 'Pts Won': 0, 'Share': 0}, inplace = True)

# Remover registros onde o time é 'TOT'
df = df[df.Team != 'TOT']

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19111 entries, 0 to 20951
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Player             19111 non-null  object
 1   Pos                19111 non-null  object
 2   Age                19111 non-null  object
 3   Team               19111 non-null  object
 4   G                  19111 non-null  object
 5   GS                 19111 non-null  object
 6   MP                 19111 non-null  object
 7   FG%                19111 non-null  object
 8   3P%                19111 non-null  object
 9   2P%                19111 non-null  object
 10  eFG%               19111 non-null  object
 11  FT%                19111 non-null  object
 12  TRB                19111 non-null  object
 13  AST                19111 non-null  object
 14  STL                19111 non-null  object
 15  BLK                19111 non-null  object
 16  TOV                19111 non-null  objec

In [ ]:
# Converter o tipo dos dados
int_columns = ['Age', 'G', 'GS', 'First', 'Pts Won', 'Team Overall Seed']
percentage_float_columns = ['FG%', '3P%', '2P%', 'eFG%', 'FT%', 'TS%', 'WS/48', 'Share']
float_columns = ['MP', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PER', 'USG%', 'WS', 'BPM', 'VORP']

for column in int_columns:
    if column == 'GS':
        df[column] = df[column].astype(str).apply(lambda x: '0' if x == '' else x)

    if column == 'First' or column == 'Pts Won':
        df[column] = df[column].astype(str).apply(lambda x: x[:-2] if '.' in x else x)

    df[column] = df[column].astype(int)

for column in percentage_float_columns:
    if column == 'Share':
        df[column] = df[column].astype(str).apply(lambda x: '0.0' if x == '0' else x)
    else:
        df[column] = df[column].apply(lambda x: x.zfill(1))
    
for column in float_columns:
    if column == 'PER' or column == 'USG%':
        df[column] = df[column].astype(str).apply(lambda x: '0.0' if x == '' else x)

    df[column] = df[column].astype(float)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19111 entries, 0 to 20951
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Player             19111 non-null  object 
 1   Pos                19111 non-null  object 
 2   Age                19111 non-null  int64  
 3   Team               19111 non-null  object 
 4   G                  19111 non-null  int64  
 5   GS                 19111 non-null  int64  
 6   MP                 19111 non-null  float64
 7   FG%                19111 non-null  object 
 8   3P%                19111 non-null  object 
 9   2P%                19111 non-null  object 
 10  eFG%               19111 non-null  object 
 11  FT%                19111 non-null  object 
 12  TRB                19111 non-null  float64
 13  AST                19111 non-null  float64
 14  STL                19111 non-null  float64
 15  BLK                19111 non-null  float64
 16  TOV                191

In [ ]:
# Exportar dataset
from google.colab import drive
drive.mount('drive')

df.to_csv('df_1980_2020.csv', index = False)
!cp df_1980_2020.csv 'drive/My Drive/'

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
